In [2]:
import os
import sys
import re
import numpy as np
import pickle
from scipy.optimize import fmin
import multiprocessing as mp
import functools

In [3]:
%load_ext autoreload
%autoreload 2

In [4]:
# statmech library setup
sys.path.append('../../statmechlib')
from statmechlib.preprocessing import select_nodes, to_param_dict, to_param_list, insert_zero_params, rescale_manybody_params
from statmechlib.preprocessing import Trajectory, scale_configuration, pair_dist_cutoff, find_index
from statmechlib.forcefields import sd2_loss, utot_EAM_per_atom, utot_EAM_per_box, ftot_EAM, udif_print, u_core
from statmechlib.preprocessing import universal_eos
from statmechlib.preprocessing import pair_dist, force_targ, get_stats_EAM_per_atom, get_stats_EAM_per_box

In [5]:
target_raw = '../data/target_raw'
target_proc = '../data/target_processed'
working = '../data/working'

In [6]:
# stats_file = 'stats_marinica' # 'stats_samples'
# target_file = 'target_marinica' # 'target_samples'
params_file = 'marinica_params'
# stats_file = 'stats_manyknots' # 'stats_samples'
# target_file = 'target_manyknots' # 'target_samples'
# stats_file = 'stats_manymd' # 'stats_samples'
# target_file = 'target_manymd' # 'target_samples'
stats_file = 'stats_manyall' # 'stats_samples'
target_file = 'target_manyall' # 'target_samples'
# stats_file = 'stats_everything' # 'stats_samples'
# target_file = 'target_everything' # 'target_samples'
stats_md_file = 'stats_onlymd' # 'stats_samples'
target_md_file = 'target_onlymd' # 'stats_samples'

In [7]:
with open(os.path.join(working, stats_file+'.pickle'), 'rb') as fi:
    stats_all = pickle.load(fi)

with open(os.path.join(working, target_file+'.pickle'), 'rb') as fi:
    targets = pickle.load(fi)
    
with open(os.path.join(working, stats_md_file+'.pickle'), 'rb') as fi:
    stats_md = pickle.load(fi)
    
with open(os.path.join(working, target_md_file+'.pickle'), 'rb') as fi:
    targets_md = pickle.load(fi)

with open(os.path.join(working, params_file + '.pickle'), 'rb') as fi:
    param_list = pickle.load(fi)  

In [8]:
def downselect(stats_inp, pair_knots, edens_knots):
    
    # find idices of knots
    pair_index = find_index(pair_knots, stats_inp['hyperparams']['pair'])
    edens_index = find_index(edens_knots, stats_inp['hyperparams']['edens'])
    
    # create boolean arrays with select indices set to True
    p_ix = np.array([True if i in pair_index else False for i in range(len(stats_inp['hyperparams']['pair']))])
    m_ix = np.array([True if i in edens_index else False for i in range(len(stats_inp['hyperparams']['edens']))])

    stats_out = select_nodes(stats_inp, p_ix, m_ix)
    
    return stats_out

In [20]:
# pair_knots = [2.22, 2.46, 2.5648975,  2.629795 ,  2.6946925,  2.8663175,  2.973045 ,
#         3.0797725,  3.5164725,  3.846445 ,  4.1764175,  4.700845 ,
#         4.8953   ,  5.089755 ,  5.3429525,  5.401695 ,  5.4604375]
# edens_knots = [ 2.5, 2.8663175, 3.1,  3.5,  4.5]

# pair_knots = [2.22, 2.46, 2.5648975,  2.629795 ,  2.6946925,  2.8663175,  2.973045 ,
#         3.0797725,  3.5164725,  3.846445 ,  4.1764175,  4.700845 ,
#         4.8953   ,  5.089755 ,  5.3429525,  5.401695 ,  5.4604375]

pair_knots = [2.22, 2.46, 2.5648975, 2.629795, 2.6946925, 2.74, 2.78, 2.8663175, 2.973045,
              3.0797725, 3.5164725, 3.846445, 4.1764175, 4.700845,
              4.8953, 5.089755, 5.3429525, 5.401695, 5.4604375]
edens_knots = [ 2.22, 2.5,  2.6946925, 2.8663175, 3.1,  3.5,  4.62]

In [21]:
stats = downselect(stats_all, pair_knots, edens_knots)
print(stats['hyperparams'])
stats_select_md = downselect(stats_md, pair_knots, edens_knots)
print(stats_select_md['hyperparams'])

{u'pair': [2.2200000000000002, 2.46, 2.5648974999999998, 2.6297950000000001, 2.6946924999999999, 2.7400000000000002, 2.7799999999999998, 2.8663175000000001, 2.9730449999999999, 3.0797724999999998, 3.5164724999999999, 3.8464450000000001, 4.1764175000000003, 4.7008450000000002, 4.8952999999999998, 5.0897550000000003, 5.3429525, 5.4016950000000001, 5.4604375000000003], u'edens': [2.2200000000000002, 2.5, 2.6946924999999999, 2.8663175000000001, 3.1000000000000001, 3.5, 4.6200000000000001]}
{u'pair': [2.2200000000000002, 2.46, 2.5648974999999998, 2.6297950000000001, 2.6946924999999999, 2.7400000000000002, 2.7799999999999998, 2.8663175000000001, 2.9730449999999999, 3.0797724999999998, 3.5164724999999999, 3.8464450000000001, 4.1764175000000003, 4.7008450000000002, 4.8952999999999998, 5.0897550000000003, 5.3429525, 5.4016950000000001, 5.4604375000000003], u'edens': [2.2200000000000002, 2.5, 2.6946924999999999, 2.8663175000000001, 3.1000000000000001, 3.5, 4.6200000000000001]}


In [22]:
targets['md'] = targets_md['md']
stats['md'] = stats_select_md['md']

In [23]:
# for key in sorted(targets.keys()):
#     for box, xyz, ene in zip(targets[key]['box'], targets[key]['xyz'], stats[key]['energy']):
#         n = len(xyz)
#         vol = np.linalg.det(box)
#         corr = float(n*n)/vol
#         ene.append([corr])

In [24]:
targets['eos_fcc']['weight'] = 0.0
targets['bain']['weight'] = 0.0
targets['eos']['weight'] = 0.0
targets['eos_bcc']['weight'] = 0.1
targets['relax']['weight'] = 0.0
#targets['liq_5000K']['weight'] = 0.1

In [25]:
targets['eos_bcc']['box'] = targets['eos_bcc']['box'][0:5]
targets['eos_bcc']['xyz'] = targets['eos_bcc']['xyz'][0:5]
targets['eos_bcc']['temp'] = targets['eos_bcc']['temp'][0:5]
targets['eos_bcc']['beta'] = targets['eos_bcc']['beta'][0:5]
targets['eos_bcc']['energy'] = targets['eos_bcc']['energy'][0:5]
targets['eos_bcc']['forces'] = targets['eos_bcc']['forces'][0:5]
stats['eos_bcc']['energy'] = stats['eos_bcc']['energy'][0:5]
stats['eos_bcc']['forces'] = stats['eos_bcc']['forces'][0:5]

In [26]:
targets.keys()

['bcc_npt_langevin_3700K',
 'md',
 'vac_npt_langevin_2000K',
 'relax',
 'screw_111_npt_langevin_2000K',
 'bcc_npt_langevin_300K',
 'i111_npt_langevin_2000K',
 'eos',
 'fcc_npt_langevin_300K',
 'eos_fcc',
 'i110_npt_langevin_2000K',
 'liq_3700K',
 'liq_5000K',
 'eos_bcc',
 'bain']

In [27]:
[(key, targets[key]['weight']) for key in targets.keys()]

[('bcc_npt_langevin_3700K', 1.0),
 ('md', 1.0),
 ('vac_npt_langevin_2000K', 1.0),
 ('relax', 0.0),
 ('screw_111_npt_langevin_2000K', 1.0),
 ('bcc_npt_langevin_300K', 1.0),
 ('i111_npt_langevin_2000K', 1.0),
 ('eos', 0.0),
 ('fcc_npt_langevin_300K', 1.0),
 ('eos_fcc', 0.0),
 ('i110_npt_langevin_2000K', 1.0),
 ('liq_3700K', 1.0),
 ('liq_5000K', 1.0),
 ('eos_bcc', 0.1),
 ('bain', 0.0)]

In [28]:
start_params = []
# start_params.append([ -4.18353755e+00,  -9.63668936e-03,  0.0, -1.00171035e+02,
#          1.90025394e+01,   2.15938317e+01,  -1.39759833e+01,
#          2.16332289e+01,  -3.29542126e+00,   1.70455674e+00,
#          1.41347064e+00,  -9.02958785e-01,  -8.62309098e-01,
#          1.95964588e+00,  -8.70527088e-01,   3.22342700e-02,
#         -1.53866121e+00,   1.37095441e+00,  -4.32896107e+01,
#          4.64461212e+00,   3.23329113e-01,   5.82061842e-02, -0.1])
# start_params.append([ -4.36548916e+00,  -1.11867827e-02,  -1.94093020e+01,
#         -1.40195282e+02,   4.53864231e+01,   1.49756180e+01,
#         -1.36031385e+01,   2.21581200e+01,  -4.61501041e+00,
#          2.06360250e+00,   1.12795645e+00,  -8.01200744e-01,
#         -8.58426028e-01,   1.95860157e+00,  -8.68817546e-01,
#          2.43936831e-02,  -1.54555297e+00,   1.38005891e+00,
#         -5.22754243e+01,   3.98209114e+00,   3.07369702e-01,
#          5.51658454e-02,   1.58601447e+00])
# start_params.append([ -4.16739003e+00,  -4.97036785e-03,  -4.62426912e+02,
#          2.63469650e+02,   1.00253629e+01,   2.05151925e+01,
#         -1.36702712e+01,   2.29129977e+01,  -1.27210473e+01,
#          2.95533783e+00,   8.18470873e-01,  -7.89494754e-01,
#         -8.74455389e-01,   2.10177842e+00,  -9.22682795e-01,
#         -6.93852493e-02,  -1.52166575e+00,   1.40766925e+00,
#          1.52571934e+02,   3.37536873e-01,   6.89310740e-01,
#          4.87102227e-02,   2.79573955e+00])
# start_params.append([ -4.33954858e+00,  -1.02215197e-02,  0.0, -1.12590032e+02,
#          3.06702045e+01,   1.74951979e+01,  -1.33298120e+01,
#          2.13899957e+01,  -3.70881631e+00,   1.56452640e+00,
#          1.27789071e+00,  -8.54623702e-01,  -8.39818015e-01,
#          1.95990543e+00,  -8.75948596e-01,   1.81927625e-02,
#         -1.54683863e+00,   1.38614460e+00,  -4.28747460e+01,
#          4.18974818e+00,   1.56622590e-01,   5.72961538e-02,
#          3.46319317e+00])

# start_params.append([ -4.41712815e+00,  -7.57807965e-03, 0.0, 2.79276814e+02,
#          1.74479561e+02,  -6.59620942e+01,   3.53977481e+01,
#         -1.40546374e+01,   2.11913467e+01,  -4.95603366e+00,
#          8.26392820e-01,   1.26391280e+00,  -9.48799563e-01,
#         -7.79099318e-01,   2.00268261e+00,  -8.90495881e-01,
#         -8.59210993e-03,  -1.54948191e+00,   1.39577855e+00,
#          2.43904610e+02,   3.47694754e+00,  -3.14999583e-01,
#          6.33069119e-02,   5.60287637e+00])

# start_params.append([-4.48347382e+00,  -7.78079006e-03,   2.92055760e+02,
#          1.84809430e+02,  -6.18705307e+01,   3.27735461e+01,
#         -1.36588262e+01,   2.11547368e+01,  -5.45394162e+00,
#          8.02998352e-01,   1.21949599e+00,  -9.43034331e-01,
#         -7.71285316e-01,   2.00564540e+00,  -8.93039729e-01,
#         -9.92574423e-03,  -1.55106037e+00,   1.39898785e+00,
#          2.54912062e+02,   3.19484582e+00,  -3.49947883e-01,
#          6.32970386e-02,   4.13612869e+00])

# start_params.append([ -4.41712815e+00,  -7.57807965e-03, 0.0, 2.79276814e+02,
#          1.74479561e+02,  -6.59620942e+01,   3.53977481e+01,
#         -1.40546374e+01,   2.11913467e+01,  -4.95603366e+00,
#          8.26392820e-01,   1.26391280e+00,  -9.48799563e-01,
#         -7.79099318e-01,   2.00268261e+00,  -8.90495881e-01,
#         -8.59210993e-03,  -1.54948191e+00,   1.39577855e+00,
#          2.43904610e+02,   3.47694754e+00,  -3.14999583e-01,
#          6.33069119e-02])

# start_params.append([ -3.09187534e+00,   1.19622206e-04,  -5.67875917e+02,
#          3.21802084e+02,  -4.71317094e+02,   3.98296604e+02,
#         -1.13270858e+02,   6.03114401e+00,   1.48067908e+01,
#         -1.57531660e+01,   7.80863462e+00,  -2.99246651e+00,
#          7.33596451e-01,  -9.50794219e-01,   1.94186078e+00,
#         -8.83596676e-01,  -1.33608784e-01,  -1.46694544e+00,
#          1.42447434e+00,   7.53865380e+01,   3.29733034e-01,
#          2.18327852e+00,   6.63152141e-02, 0.0])

# start_params.append([ -2.95016477e+00,   3.78177969e-04,  -5.21095611e+02,
#          3.09845244e+02,  -4.69222440e+02,   3.95186037e+02,
#         -1.11630659e+02,   5.36915127e+00,   1.52984702e+01,
#         -1.54915170e+01,   7.53107481e+00,  -2.98591742e+00,
#          7.55346307e-01,  -1.00678488e+00,   1.96634582e+00,
#         -8.71560927e-01,  -1.41820832e-01,  -1.46508000e+00,
#          1.42936365e+00,   7.23805795e+01,   0.0, 9.18071460e-01,
#          2.04259992e+00,   6.28575567e-02,   7.62804667e-03])

# start_params.append([ -3.27882058e+00,  -9.77681860e-04,  -5.95486537e+02,
#          3.42401535e+02,  -4.15497479e+02,   3.71316746e+02,
#         -1.07274610e+02,  -2.93282898e+00,   1.51298572e+01,
#         -1.37454674e+01,   8.16724633e+00,  -3.16850352e+00,
#          8.74607042e-01,  -1.07878768e+00,   1.97939820e+00,
#         -8.26149646e-01,  -1.62437297e-01,  -1.45694910e+00,
#          1.42153527e+00,   1.29203135e+02,  -9.25284676e+00,
#          2.63967417e+00,   2.06410900e+00,   6.03258240e-02*2])

# start_params.append([ -3.56709315e+00,  -2.97358686e-03,  -4.37632666e+02,
#          3.35095381e+02,  -2.95083648e+02,   2.94263473e+02,
#         -8.57731610e+01,  -1.62036228e+01,   1.30838141e+01,
#         -6.46551376e+00,   8.01826587e+00,  -3.58786095e+00,
#          1.25735987e+00,  -1.21627097e+00,   2.04033545e+00,
#         -8.04702913e-01,  -1.92666480e-01,  -1.44083371e+00,
#          1.41816098e+00,   1.70360405e+02,  -1.89070431e+01,
#          7.47139804e+00,   1.20540194e+00,   1.34588181e-01, 0.0])

# start_params.append([ -3.82648588e+00,  -3.15395324e-03,  -7.04235860e+02,
#          2.24162391e+02,   6.00323185e+01,  -2.50405997e+01,
#          3.08538238e+01,  -1.21719898e+01,   1.66904242e+01,
#         -3.84273118e+00,   8.34954792e-01,   1.12083715e+00,
#         -7.94620177e-01,  -6.05852952e-01,   2.03652986e+00,
#         -1.03495446e+00,  -1.20820798e-01,  -1.55518420e+00,
#          1.50033948e+00,   2.09695356e+02,   1.05734976e-02,
#          3.52470642e+00,  -2.47034958e-01,   1.52735787e-01, 0.0])

# start_params.append([ -3.91050922e+00,  -3.58200201e-03,  -6.99584889e+02,
#          2.33989111e+02,   1.04132222e+02,  -5.69974715e+01,
#          4.22990605e+01,  -1.67015822e+01,   1.54096896e+01,
#         -7.00711322e-01,   4.94371835e-01,   1.05428858e+00,
#         -7.24094984e-01,  -6.38208084e-01,   2.09639444e+00,
#         -1.06529597e+00,  -1.90927102e-01,  -1.53841395e+00,
#          1.54612418e+00,   2.28170241e+02,  -2.63164086e+00,
#          4.77073224e+00,  -4.55727595e-01,   1.53656127e-01,
#         -2.46946053e+00])

# start_params.append([ -2.84299893e+00,   7.74316066e-04,  -2.60185762e+02,
#          2.12100147e+01,   3.52845699e+01,  -6.93103301e+01,
#          7.95717459e+01,  -3.39400457e+01,   9.42654140e+00,
#          5.61056055e+00,   2.45706583e+00,   2.28984411e-01,
#         -7.34032706e-01,  -8.98635826e-01,   2.20701099e+00,
#         -7.55061461e-01,  -2.30027272e-01,  -1.58424621e+00,
#          1.47015547e+00,   5.39975665e+01,   5.25810283e+01,
#          4.00257405e+01,  -2.81864073e+01,   1.39508804e+01,
#          1.74493416e-01,   1.98910760e-01])

# start_params.append([ -3.34519702e+00,   7.66822271e-04,  -4.03553412e+02,
#          1.55282185e+01,   2.90516401e+01,  -6.23087513e+01,
#          8.48184009e+01,  -3.48806112e+01,   9.13550274e+00,
#          5.96096071e+00,   2.54062374e+00,   2.39873080e-01,
#         -7.19239652e-01,  -1.04267291e+00,   2.29366373e+00,
#         -7.05206149e-01,  -2.72279799e-01,  -1.55954882e+00,
#          1.45964251e+00,  -2.01431222e+02,   3.24297105e+01,
#          3.81664135e+01,  -2.18948938e+01,   1.02484895e+01,
#          2.20300623e-01,   1.13874983e-01])

# start_params.append([-3.36853915e+00,   8.06754487e-04,  6.27504135e+02,   1.84765652e+00,   6.61989449e+01,
#         -7.80107146e+01,   8.89234774e+01,  -1.54057338e-02,
#         -3.52908615e+01,   7.56470236e+00,   6.96286556e+00,
#          2.40847520e+00,   3.99185717e-01,  -9.49757520e-01,
#         -8.51778757e-01,   3.14384571e+00,  -2.51275693e+00,
#          5.32526064e-01,  -8.45163668e-03, 43.3363673 ,  60.4909283 ,  40.4214288 , -23.4985547 ,
#         10.3464548 ,   0.209124  , 0.11044082])

# start_params.append([ -3.34519702e+00,   7.66822271e-04,  -4.03553412e+02,
#          1.55282185e+01,   2.90516401e+01,  -6.23087513e+01, 
#          8.48184009e+01,  0.0, -3.48806112e+01,   9.13550274e+00,
#          5.96096071e+00,   2.54062374e+00,   2.39873080e-01,
#         -7.19239652e-01,  -1.04267291e+00,   2.29366373e+00,
#         -7.05206149e-01,  -2.72279799e-01,  -1.55954882e+00,
#          1.45964251e+00,  -2.01431222e+02,   3.24297105e+01,
#          3.81664135e+01,  -2.18948938e+01,   1.02484895e+01,
#          2.20300623e-01,   1.13874983e-01])

# start_params.append([ -3.28450559e+00,   5.83025682e-04,  -4.17316283e+02,
#         -2.95796266e+00,   9.19216933e+01,  -1.31503227e+02,
#          1.39561590e+02,  -3.46555386e+01,  0.0, -1.92301185e+01,
#          2.73677719e+00,   5.04115541e+00,   2.91028470e+00,
#          4.13914057e-01,  -8.16902340e-01,  -1.06905912e+00,
#          2.30104311e+00,  -6.78279770e-01,  -2.58998819e-01,
#         -1.56682263e+00,   1.44421769e+00,  -2.21658364e+02,
#          5.03420490e+01,   3.42733093e+01,  -1.77132859e+01,
#          7.87963302e+00,   8.32179244e-01,   1.01343434e-01])

# start_params.append([ -6.76671347e+00,   4.10083930e-03,  -7.49159809e+02,
#          3.44690642e+02,  -5.22145270e+02,   4.55738400e+02,
#         -1.32915886e+02,   8.17203560e+00,   2.15129227e+00,
#          1.12103814e+00,   3.31607800e+00,  -1.64662673e+00,
#          1.87730747e-01,  -5.41261773e-01,   1.94159548e+00,
#         -1.16256235e+00,  -1.07742642e-01,  -1.52176937e+00,
#          1.55975528e+00,   2.20884953e+01,  -7.92970300e-01,
#          2.00236877e+00,  -2.30635587e-01,   5.45564868e-02,
#         -4.76235960e+01])

start_params.append([-3.24299854e+00,   5.32903957e-04,  -3.71157162e+02,
        -3.61968049e+00,   8.67969153e+01,  -1.31614186e+02,
         1.40679601e+02,  -4.00348028e+01,   2.25915962e+00,
        -1.51102444e+01,   2.74172354e+00,   3.44463905e+00,
         3.00556344e+00,   5.16980263e-01,  -8.60974907e-01,
        -1.07980283e+00,   2.30139168e+00,  -6.66175433e-01,
        -2.54147622e-01,  -1.56760451e+00,   1.43704165e+00,
        -1.76785542e+02,   4.32488872e+01,   3.18295058e+01,
        -1.38529740e+01,   6.41981570e+00,   1.09607439e+00,
         9.87706840e-02])

multi_pars = start_params
print(multi_pars)

[[-3.24299854, 0.000532903957, -371.157162, -3.61968049, 86.7969153, -131.614186, 140.679601, -40.0348028, 2.25915962, -15.1102444, 2.74172354, 3.44463905, 3.00556344, 0.516980263, -0.860974907, -1.07980283, 2.30139168, -0.666175433, -0.254147622, -1.56760451, 1.43704165, -176.785542, 43.2488872, 31.8295058, -13.852974, 6.4198157, 1.09607439, 0.098770684]]


In [29]:
def optimize_EAM_mp(pars, targets, stats, utot_func):
    
    output = fmin(sd2_loss, pars, args=(targets, stats, utot_EAM_per_atom, None, 0.05, 1), maxiter=100000, maxfun=100000, disp=0, full_output=1,ftol=1e-6)

    return tuple([output[1], output[0]])

In [ ]:
params_output = []

pool = mp.Pool()

for ieam in range(1):#5, len(stats_opts[it]['hyperparams'])):

    get_sd = functools.partial(sd2_loss, targets=targets, stats=stats, utot_func=utot_EAM_per_atom, ftot_func=None, dl=0.05, verbose=1)

    get_opt = functools.partial(optimize_EAM_mp, targets=targets, stats=stats, utot_func=utot_EAM_per_atom)

    # initial ordering
    print('Initial params:', multi_pars[-1])
    
    #distances = pool.map(get_sd, multi_pars)
    
    distances = [get_sd(multi_pars[-1])]
    
    optimal_params = zip(distances, multi_pars[-1:])
    best_params = sorted(optimal_params, key=lambda param: param[0], reverse=True)
    m_pars = [p[1] for p in best_params]

    print('Best params:', best_params)
    print('ieam:', ieam)
    #m_pars = multi_pars

    for i in range(50):
        #optimal_params = pool.map(get_opt, m_pars)
        optimal_params = [get_opt(m_pars)]
        best_params = sorted(optimal_params, key=lambda param: param[0], reverse=True)
        m_pars = [p[1] for p in best_params]
        
        print("Iteration {}, best params: {}".format(i, best_params))

    print('Final Best params:', best_params)
    
    params_output.append(best_params)

pool.close()
pool.join()

('Initial params:', [-3.24299854, 0.000532903957, -371.157162, -3.61968049, 86.7969153, -131.614186, 140.679601, -40.0348028, 2.25915962, -15.1102444, 2.74172354, 3.44463905, 3.00556344, 0.516980263, -0.860974907, -1.07980283, 2.30139168, -0.666175433, -0.254147622, -1.56760451, 1.43704165, -176.785542, 43.2488872, 31.8295058, -13.852974, 6.4198157, 1.09607439, 0.098770684])
loss 0.00238353315234 0.00238353315234 0.0
('Best params:', [(0.0023835331523364501, [-3.24299854, 0.000532903957, -371.157162, -3.61968049, 86.7969153, -131.614186, 140.679601, -40.0348028, 2.25915962, -15.1102444, 2.74172354, 3.44463905, 3.00556344, 0.516980263, -0.860974907, -1.07980283, 2.30139168, -0.666175433, -0.254147622, -1.56760451, 1.43704165, -176.785542, 43.2488872, 31.8295058, -13.852974, 6.4198157, 1.09607439, 0.098770684])])
('ieam:', 0)
loss 0.00238353315234 0.00238353315234 0.0
loss 0.00277827213412 0.00277827213412 0.0
loss 0.00243441371472 0.00243441371472 0.0
loss 0.00238353325393 0.00238353325

loss 0.00154237737193 0.00154237737193 0.0
loss 0.00138696953335 0.00138696953335 0.0
loss 0.00162925826839 0.00162925826839 0.0
loss 0.00143895947887 0.00143895947887 0.0
loss 0.00151097256889 0.00151097256889 0.0
loss 0.00150148362665 0.00150148362665 0.0
loss 0.00151203309252 0.00151203309252 0.0
loss 0.00143224243664 0.00143224243664 0.0
loss 0.00143894518071 0.00143894518071 0.0
loss 0.00172316614996 0.00172316614996 0.0
loss 0.00149690065362 0.00149690065362 0.0
loss 0.00143383137213 0.00143383137213 0.0
loss 0.00135393625123 0.00135393625123 0.0
loss 0.00143229656426 0.00143229656426 0.0
loss 0.0014443707966 0.0014443707966 0.0
loss 0.00153597843263 0.00153597843263 0.0
loss 0.00139017372127 0.00139017372127 0.0
loss 0.00139958019902 0.00139958019902 0.0
loss 0.00140676989548 0.00140676989548 0.0
loss 0.00149588255454 0.00149588255454 0.0
loss 0.00138130722175 0.00138130722175 0.0
loss 0.00156767299429 0.00156767299429 0.0
loss 0.00138683575462 0.00138683575462 0.0
loss 0.001411

loss 0.000816977519303 0.000816977519303 0.0
loss 0.000859134802818 0.000859134802818 0.0
loss 0.000890293520121 0.000890293520121 0.0
loss 0.000722900372646 0.000722900372646 0.0
loss 0.000627875926111 0.000627875926111 0.0
loss 0.00078325472158 0.00078325472158 0.0
loss 0.000763904580665 0.000763904580665 0.0
loss 0.000775924310614 0.000775924310614 0.0
loss 0.000839082175469 0.000839082175469 0.0
loss 0.000812697558033 0.000812697558033 0.0
loss 0.000726154813126 0.000726154813126 0.0
loss 0.000669519132938 0.000669519132938 0.0
loss 0.000728532793506 0.000728532793506 0.0
loss 0.000692096820601 0.000692096820601 0.0
loss 0.00075799494671 0.00075799494671 0.0
loss 0.000691210218445 0.000691210218445 0.0
loss 0.000650029897439 0.000650029897439 0.0
loss 0.000776980247077 0.000776980247077 0.0
loss 0.000629938824228 0.000629938824228 0.0
loss 0.000623961047076 0.000623961047076 0.0
loss 0.000561643022235 0.000561643022235 0.0
loss 0.000634254120215 0.000634254120215 0.0
loss 0.0006044

loss 0.000259885473966 0.000259885473966 0.0
loss 0.000257244457849 0.000257244457849 0.0
loss 0.00025668586933 0.00025668586933 0.0
loss 0.000257247507495 0.000257247507495 0.0
loss 0.000285925653642 0.000285925653642 0.0
loss 0.000264891145546 0.000264891145546 0.0
loss 0.000282905933927 0.000282905933927 0.0
loss 0.000264550304478 0.000264550304478 0.0
loss 0.000254769331926 0.000254769331926 0.0
loss 0.000253931469561 0.000253931469561 0.0
loss 0.000264811281372 0.000264811281372 0.0
loss 0.000262229101748 0.000262229101748 0.0
loss 0.000302607113876 0.000302607113876 0.0
loss 0.000259443679559 0.000259443679559 0.0
loss 0.000265411376054 0.000265411376054 0.0
loss 0.000256597037074 0.000256597037074 0.0
loss 0.000268136852764 0.000268136852764 0.0
loss 0.000260941020031 0.000260941020031 0.0
loss 0.000250591989817 0.000250591989817 0.0
loss 0.000247289932432 0.000247289932432 0.0
loss 0.000258433969897 0.000258433969897 0.0
loss 0.000236616444748 0.000236616444748 0.0
loss 0.00022

loss 0.000192069753514 0.000192069753514 0.0
loss 0.000197057093515 0.000197057093515 0.0
loss 0.000185528535971 0.000185528535971 0.0
loss 0.000185163915002 0.000185163915002 0.0
loss 0.0001878665857 0.0001878665857 0.0
loss 0.000186941499893 0.000186941499893 0.0
loss 0.000194598423637 0.000194598423637 0.0
loss 0.000195563166926 0.000195563166926 0.0
loss 0.000196179052767 0.000196179052767 0.0
loss 0.000196107420782 0.000196107420782 0.0
loss 0.000187326277897 0.000187326277897 0.0
loss 0.000194126875108 0.000194126875108 0.0
loss 0.000198437475103 0.000198437475103 0.0
loss 0.00018978117173 0.00018978117173 0.0
loss 0.000189453038235 0.000189453038235 0.0
loss 0.00019032789123 0.00019032789123 0.0
loss 0.000188192917009 0.000188192917009 0.0
loss 0.000185879420626 0.000185879420626 0.0
loss 0.000198114115839 0.000198114115839 0.0
loss 0.000188943745689 0.000188943745689 0.0
loss 0.000192615918921 0.000192615918921 0.0
loss 0.00018824922904 0.00018824922904 0.0
loss 0.0001916782565

...

loss 1.91643736614e-06 1.91643736614e-06 0.0
loss 1.91643736893e-06 1.91643736893e-06 0.0
loss 1.91643736139e-06 1.91643736139e-06 0.0
loss 1.91643736905e-06 1.91643736905e-06 0.0
loss 1.91643736201e-06 1.91643736201e-06 0.0
loss 1.91643736872e-06 1.91643736872e-06 0.0
loss 1.91643736154e-06 1.91643736154e-06 0.0
loss 1.91643737318e-06 1.91643737318e-06 0.0
loss 1.91643736194e-06 1.91643736194e-06 0.0
loss 1.91643737009e-06 1.91643737009e-06 0.0
loss 1.91643735997e-06 1.91643735997e-06 0.0
loss 1.91643736882e-06 1.91643736882e-06 0.0
loss 1.91643736241e-06 1.91643736241e-06 0.0
loss 1.91643736703e-06 1.91643736703e-06 0.0
loss 1.91643736154e-06 1.91643736154e-06 0.0
loss 1.91643736991e-06 1.91643736991e-06 0.0
loss 1.91643736114e-06 1.91643736114e-06 0.0
loss 1.9164373698e-06 1.9164373698e-06 0.0
loss 1.91643736119e-06 1.91643736119e-06 0.0
loss 1.91643736987e-06 1.91643736987e-06 0.0
loss 1.91643736263e-06 1.91643736263e-06 0.0
loss 1.91643736194e-06 1.91643736194e-06 0.0
loss 1.91643

loss 1.91643735781e-06 1.91643735781e-06 0.0
loss 1.91643735937e-06 1.91643735937e-06 0.0
loss 1.91643735897e-06 1.91643735897e-06 0.0
loss 1.9164373597e-06 1.9164373597e-06 0.0
loss 1.91643736012e-06 1.91643736012e-06 0.0
loss 1.91643735903e-06 1.91643735903e-06 0.0
loss 1.91643736005e-06 1.91643736005e-06 0.0
loss 1.9164373597e-06 1.9164373597e-06 0.0
loss 1.91643735941e-06 1.91643735941e-06 0.0
loss 1.91643735941e-06 1.91643735941e-06 0.0
loss 1.91643735857e-06 1.91643735857e-06 0.0
loss 1.91643735861e-06 1.91643735861e-06 0.0
loss 1.91643735877e-06 1.91643735877e-06 0.0
loss 1.9164373589e-06 1.9164373589e-06 0.0
loss 1.91643735943e-06 1.91643735943e-06 0.0
loss 1.91643735806e-06 1.91643735806e-06 0.0
loss 1.91643735817e-06 1.91643735817e-06 0.0
loss 1.91643735826e-06 1.91643735826e-06 0.0
loss 1.9164373597e-06 1.9164373597e-06 0.0
loss 1.91643735826e-06 1.91643735826e-06 0.0
loss 1.91643735868e-06 1.91643735868e-06 0.0
loss 1.9164373591e-06 1.9164373591e-06 0.0
loss 1.91643735903e-

loss 1.9164373573e-06 1.9164373573e-06 0.0
loss 1.91643735979e-06 1.91643735979e-06 0.0
loss 1.91643735905e-06 1.91643735905e-06 0.0
loss 1.91643735965e-06 1.91643735965e-06 0.0
loss 1.91643735959e-06 1.91643735959e-06 0.0
loss 1.91643735872e-06 1.91643735872e-06 0.0
loss 1.91643735974e-06 1.91643735974e-06 0.0
loss 1.9164373601e-06 1.9164373601e-06 0.0
loss 1.91643735897e-06 1.91643735897e-06 0.0
loss 1.91643735961e-06 1.91643735961e-06 0.0
loss 1.9164373583e-06 1.9164373583e-06 0.0
loss 1.91643735945e-06 1.91643735945e-06 0.0
loss 1.91643735985e-06 1.91643735985e-06 0.0
loss 1.91643735799e-06 1.91643735799e-06 0.0
loss 1.91643735932e-06 1.91643735932e-06 0.0
loss 1.91643735888e-06 1.91643735888e-06 0.0
loss 1.91643735837e-06 1.91643735837e-06 0.0
loss 1.91643735668e-06 1.91643735668e-06 0.0
loss 1.91643735894e-06 1.91643735894e-06 0.0
loss 1.91643735828e-06 1.91643735828e-06 0.0
loss 1.91643735919e-06 1.91643735919e-06 0.0
loss 1.91643735823e-06 1.91643735823e-06 0.0
loss 1.916437359

loss 1.91643735772e-06 1.91643735772e-06 0.0
loss 1.91643735905e-06 1.91643735905e-06 0.0
loss 1.9164373587e-06 1.9164373587e-06 0.0
loss 1.91643735812e-06 1.91643735812e-06 0.0
loss 1.91643735699e-06 1.91643735699e-06 0.0
loss 1.91643735919e-06 1.91643735919e-06 0.0
loss 1.91643735859e-06 1.91643735859e-06 0.0
loss 1.91643735925e-06 1.91643735925e-06 0.0
loss 1.91643735814e-06 1.91643735814e-06 0.0
loss 1.9164373589e-06 1.9164373589e-06 0.0
loss 1.91643735941e-06 1.91643735941e-06 0.0
loss 1.91643735865e-06 1.91643735865e-06 0.0
loss 1.91643736036e-06 1.91643736036e-06 0.0
loss 1.91643735937e-06 1.91643735937e-06 0.0
loss 1.91643736016e-06 1.91643736016e-06 0.0
loss 1.91643735723e-06 1.91643735723e-06 0.0
loss 1.9164373587e-06 1.9164373587e-06 0.0
loss 1.91643735974e-06 1.91643735974e-06 0.0
loss 1.91643735766e-06 1.91643735766e-06 0.0
loss 1.91643735943e-06 1.91643735943e-06 0.0
loss 1.91643735941e-06 1.91643735941e-06 0.0
loss 1.91643735934e-06 1.91643735934e-06 0.0
loss 1.916437358

loss 0.083897063911 0.083897063911 0.0
loss 0.0272224618059 0.0272224618059 0.0
loss 0.0550889228405 0.0550889228405 0.0
loss 0.0108940151378 0.0108940151378 0.0
loss 0.0363151380959 0.0363151380959 0.0
loss 0.0112571104557 0.0112571104557 0.0
loss 0.0368594881618 0.0368594881618 0.0
loss 0.0106899262811 0.0106899262811 0.0
loss 0.0290754995728 0.0290754995728 0.0
loss 0.00796196557855 0.00796196557855 0.0
loss 0.0202310339058 0.0202310339058 0.0
loss 0.00301057213719 0.00301057213719 0.0
loss 0.00917016530325 0.00917016530325 0.0
loss 0.0134021523664 0.0134021523664 0.0
loss 0.00225269812571 0.00225269812571 0.0
loss 0.0105246155184 0.0105246155184 0.0
loss 0.00242474951467 0.00242474951467 0.0
loss 0.0118366878897 0.0118366878897 0.0
loss 0.00252318745047 0.00252318745047 0.0
loss 0.0112571687671 0.0112571687671 0.0
loss 0.00233001377438 0.00233001377438 0.0
loss 0.00857422047378 0.00857422047378 0.0
loss 0.00228254866637 0.00228254866637 0.0
loss 0.00900823032995 0.00900823032995 0.

In [ ]:
params_output = [(8.9710060363567114e-07, np.array([ -4.18353755e+00,  -9.63668936e-03,  -1.00171035e+02,
         1.90025394e+01,   2.15938317e+01,  -1.39759833e+01,
         2.16332289e+01,  -3.29542126e+00,   1.70455674e+00,
         1.41347064e+00,  -9.02958785e-01,  -8.62309098e-01,
         1.95964588e+00,  -8.70527088e-01,   3.22342700e-02,
        -1.53866121e+00,   1.37095441e+00,  -4.32896107e+01,
         4.64461212e+00,   3.23329113e-01,   5.82061842e-02]))]

In [ ]:
params_output = [(8.829397265807305e-07, np.array([ -4.33954858e+00,  -1.02215197e-02,  -1.12590032e+02,
         3.06702045e+01,   1.74951979e+01,  -1.33298120e+01,
         2.13899957e+01,  -3.70881631e+00,   1.56452640e+00,
         1.27789071e+00,  -8.54623702e-01,  -8.39818015e-01,
         1.95990543e+00,  -8.75948596e-01,   1.81927625e-02,
        -1.54683863e+00,   1.38614460e+00,  -4.28747460e+01,
         4.18974818e+00,   1.56622590e-01,   5.72961538e-02,
         3.46319317e+00]))]

In [ ]:
params_output = [(8.7421414571196454e-07, np.array([ -4.31249172e+00,  -9.93971047e-03,  -3.76018390e+01,
        -1.14759006e+02,   1.72557600e+01,   2.66437414e+01,
        -1.50817090e+01,   2.22342349e+01,  -3.62963119e+00,
         1.62230813e+00,   1.18480877e+00,  -8.52354746e-01,
        -8.38460594e-01,   1.98957701e+00,  -8.86622667e-01,
         1.66082225e-03,  -1.54824194e+00,   1.39761513e+00,
        -5.06767503e+01,   4.47029066e+00,   9.10205250e-02,
         5.87302830e-02,   1.82639066e+00]))]

In [ ]:
params_output = [(8.8257346852300666e-07, np.array([ -4.36548916e+00,  -1.11867827e-02,  -1.94093020e+01,
        -1.40195282e+02,   4.53864231e+01,   1.49756180e+01,
        -1.36031385e+01,   2.21581200e+01,  -4.61501041e+00,
         2.06360250e+00,   1.12795645e+00,  -8.01200744e-01,
        -8.58426028e-01,   1.95860157e+00,  -8.68817546e-01,
         2.43936831e-02,  -1.54555297e+00,   1.38005891e+00,
        -5.22754243e+01,   3.98209114e+00,   3.07369702e-01,
         5.51658454e-02,   1.58601447e+00]))]

In [ ]:
params_output = [(2.8925274000958436e-06, np.array([ -3.09187534e+00,   1.19622206e-04,  -5.67875917e+02,
         3.21802084e+02,  -4.71317094e+02,   3.98296604e+02,
        -1.13270858e+02,   6.03114401e+00,   1.48067908e+01,
        -1.57531660e+01,   7.80863462e+00,  -2.99246651e+00,
         7.33596451e-01,  -9.50794219e-01,   1.94186078e+00,
        -8.83596676e-01,  -1.33608784e-01,  -1.46694544e+00,
         1.42447434e+00,   7.53865380e+01,   3.29733034e-01,
         2.18327852e+00,   6.63152141e-02]))]

In [ ]:
start_params

In [ ]:
params_output = start_params
# convert lists of optimal parameters to dictionaries
params_optimal = []
for pars in params_output:
    #params_optimal.append(to_param_dict(pars, [2, 15, 4, 0]))
    params_optimal.append(to_param_dict(pars[1], stats['hyperparams']))

In [ ]:
# store hyperparameters and output parameters in a single file
try:
    with open(os.path.join(working, 'params_store'+'.pickle'), 'rb') as fi:
        params_store = pickle.load(fi)    
except IOError:
    print('no file')
    params_store = []
    
params_store.extend(params_optimal)

with open(os.path.join(working, "params_store.pickle"), 'wb') as fo:
    pickle.dump(params_store, fo, protocol=2)

In [ ]:
with open(os.path.join(working, 'params_store' + '.pickle'), 'rb') as fi:
    retreived_params = pickle.load(fi)  

In [ ]:
retreived_params